In [1]:
!pip install lightkurve 
!pip install transitleastsquares
!pip install wotan
!pip install astroquery
import lightkurve as lk
import transitleastsquares as tls
from wotan import flatten, t14
import astroquery.mast as astromast

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 247 kB 5.1 MB/s 
     |████████████████████████████████| 4.5 MB 56.6 MB/s 
     |████████████████████████████████| 98 kB 7.5 MB/s 
     |████████████████████████████████| 41 kB 176 kB/s 
     |████████████████████████████████| 832 kB 64.2 MB/s 
     |████████████████████████████████| 48 kB 4.1 MB/s 
     |████████████████████████████████| 4.0 MB 43.6 MB/s 
  Created wheel for fbpca: filename=fbpca-1.0-py3-none-any.whl size=11375 sha256=0393cf8ef408a66d19ccd451990ca11eb365ab6346de0ab5b863acff5fedd297
  Stored in directory: /root/.cache/pip/wheels/93/08/0c/1b9866c35c8d3f136d100dfe88036a32e0795437daca089f70
  Created wheel for memoization: filename=memoization-0.4.0-py3-none-any.whl size=50467 sha256=133384cf0bb8e814d8165f0c636e6ab9fc88dbe00fe2db8f42a6b4ec0e89b682
  Stored in directory: /root/.cache/pip/wheels/38/f7/65/161985e7311dd484a23b3a5c9149995db

In [2]:
import numpy as np
import pandas as pd
import scipy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input
from astropy.io import fits
import os 
import subprocess
import matplotlib.pyplot as plt

In [3]:
import time
import tqdm
from astropy.stats import sigma_clip
import random

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
def load_file(n):
  return '/content/drive/Othercomputers/My Laptop/ML Project/MAST/tic2file-sec'+ str(n) +'.csv'

def read_csv(n):
  csv_file = load_file(n)
  df = pd.read_csv(csv_file)
  return df

# print(read_csv(21))

def light_curve(n,id):
  light_curve = lk.search_lightcurve('TIC' + str(id), exptime = 120, sector = n).download()
  return light_curve

# light_curve(21, read_csv(21)['tic_id'][0])

In [6]:
def load_lightcurve(n, tess_id):
  curves = []
  initial_path = '/content/drive/Othercomputers/My Laptop/ML Project/MAST/Sector ' + str(n) + '/'
  files = read_csv(n)['Filename'][read_csv.loc(read_csv['tic_id'] == tess_id)]
  for i in files:
    filepath = os.path.join(initial_path, i)
    curves.append(lk.search.open(filepath))
  light_curves = lk.LightCurveFileCollection(curves)
  return light_curves.stitch()

In [7]:
read_csv(24).head()

,tic_id,Sector,Filename
0,275634677,24,tess2020106103520-s0024-0000000275634677-0180-...
1,428740087,24,tess2020106103520-s0024-0000000428740087-0180-...
2,441802949,24,tess2020106103520-s0024-0000000441802949-0180-...
3,188768068,24,tess2020106103520-s0024-0000000188768068-0180-...
4,343410928,24,tess2020106103520-s0024-0000000343410928-0180-...


### 2

In [62]:
# Takes some time to compute
def fits_data(n):
  df = read_csv(n)
  initial_path = '/content/drive/Othercomputers/My Laptop/ML Project/MAST/Sector ' + str(n) + '/'
  for i in range(len(df)):
    # print(i)
    file_fit = os.path.join(initial_path, df['Filename'][i])
    if not os.path.exists(file_fit):
      continue         
    try:
      # print('ok')
      tic = T0= depth= period =uncertainty=duration=t_mag=t_eff=radius_0=distinct_transits=a=b=log=rp_rs=depth_even=depth_odd=odd_even_mismatch=snr=SDE = []
      with fits.open(file_fit) as hdu:
        hdu.info()
        tic_id = hdu[0].header['TICID']
        tess_mag = hdu[0].header["TESSMAG"] # [mag]
        temp_eff = hdu[0].header["TEFF"] # K
        logg = hdu[0].header["LOGG"] # [cm/s2] log10 surface gravity
        rad = hdu[0].header['RADIUS'] # [solar radii] stellar radius 
        time = hdu[1].data['TIME']
        flux = hdu[1].data['PDCSAP_FLUX']  
        time, flux = tls.cleaned_array(time, flux)  
        flux = flux / np.median(flux)
        ab, mass, mass_min, mass_max, radius, radius_min, radius_max = tls.catalog_info(TIC_ID= tic_id)
        print('Searching with limb-darkening estimates using quadratic LD (a,b)=', ab)

        # print('Period', format(results.period, '.5f'), 'd at T0=', results.T0)
        # print(len(results.transit_times), 'transit times in time series:', ['{0:0.5f}'.format(i) for i in results.transit_times])
        # print('Number of data points during each unique transit', results.per_transit_count)
        # print('The number of transits with intransit data points', results.distinct_transit_count)
        # print('The number of transits with no intransit data points', results.empty_transit_count)
        # print('Transit depth', format(results.depth, '.5f'), '(at the transit bottom)')
        # print('Transit duration (days)', format(results.duration, '.5f'))
        # print('Transit depths (mean)', results.transit_depths)
        # print('Transit depth uncertainties', results.transit_depths_uncertainties)
        if np.isnan(mass) or np.isnan(radius):
            flatten_lc, trend_lc = flatten(time, flux, window_length=0.5, method='biweight', return_trend=True)
            flux = sigma_clip(flatten_lc, sigma_upper=3, sigma_lower=20)
            model = tls.transitleastsquares(time, flux)
            results = model.power(u=ab,oversampling_factor=5)
        else:
            period_2 = 13.5
            tdur = t14(R_s=radius, M_s=mass, P=period_2, small_planet=False)
            flatten_lc, trend_lc = flatten(time, flux, window_length=3*tdur, method='biweight', return_trend=True)
            flux = sigma_clip(flatten_lc, sigma_upper=3, sigma_lower=20)
            model = tls.transitleastsquares(time, flux)
            mstar_min = mass - 3 * mass_min
            mstar_max = mass + 3 * mass_max
            rstar_max = radius + 3 * radius_max
            results = model.power(u=ab, M_star_min=mstar_min, M_star=mass, M_star_max=mstar_max, R_star_min=rstar_min, R_star=radius, R_star_max=rstar_max)
        tic.append(tic_id)
        T0.append(results.T0) 
        depth.append(results.depth) 
        duration.append(results.duration * 24)
        t_mag.append(tess_mag) 
        t_eff.append(temp_eff)
        log.append(logg)
        period.append(results.period)
        uncertainty.append(results.period_uncertainty)
        radius_0.append(rad)
        distinct_transits.append(results.distinct_transit_count)
        a.append(ab[0])
        b.append(ab[1])
        rp_rs.append(results.rp_rs)
        depth_even.append(results.depth_mean_even[0])
        depth_odd.append(results.depth_mean_odd[0])
        odd_even_mismatch.append(results.odd_even_mismatch)
        snr.append(results.snr)
        SDE.append(results.SDE)
    except (OSError, TypeError):
        # print('oops')
        return None
  # print(tic)
  new_dataframe = pd.DataFrame({'tic_id': tic, 'T0': T0, 'depth': depth,'duration':duration, 'tess_mag': t_mag,
                                'temp_eff': t_eff, 'logg': log, 'rp_rs': rp_rs,  'period' : period, 'period_uncertainty':uncertainty,'radius': radius_0, 
                                'distinct_transit_count': distinct_transits, 'a': a, 'b': b, 'depth_even':depth_even, 'depth_odd':depth_odd, 
                                'odd_even_mismatch':odd_even_mismatch, 'snr': snr, 'sde': SDE})
  return (new_dataframe,results)


In [63]:
# fits_data(6)[0]

https://github.com/hippke/tls/blob/master/tutorials/11%20A%20planet%20from%20TESS%20with%20TLS.ipynb 

https://github.com/hippke/wotan/blob/master/tutorials/01%20Basic%20functionality.ipynb

Wotan documentation- https://wotan.readthedocs.io/_/downloads/en/stable/pdf/

In [64]:
def stellar_params(n):
  file_csv = fits_data(n)[0]
  tic = file_csv['tic_id']
  mass = dist = logg = lum = rho = []
  for id in tic:
    # print(id)
    result = astromast.Catalogs.query_criteria(catalog='TIC', ID=tic)
    # print(result['mass'])
    mass.append(result['mass'])
    # print(id)
    dist.append(result['d'])
    lum.append(result['lum'])
    logg.append(result['logg'])
    rho.append(result['rho'])
  file_csv['Mass'] = mass
  file_csv['distance'] = dist
  file_csv['luminosity'] = lum
  file_csv['rho'] = rho
  return file_csv

In [12]:
def plot_phase(result,periodogram = False):
    if perdiodogram == True:
      plt.figure()
      ax = plt.gca()
      ax.axvline(results.period, alpha=0.4, lw=3)
      plt.xlim(numpy.min(results.periods), numpy.max(results.periods))
      for n in range(2, 10):
          ax.axvline(n*results.period, alpha=0.4, lw=1, linestyle="dashed")
          ax.axvline(results.period / n, alpha=0.4, lw=1, linestyle="dashed")
      plt.ylabel(r'SDE')
      plt.xlabel('Period (days)')
      plt.plot(results.periods, results.power, color='black', lw=0.5)
      plt.xlim(0, max(results.periods));  
    else:
      plt.figure()
      plt.plot(result.model_folded_phase, result.model_folded_model, color='red')
      plt.scatter(result.folded_phase, result.folded_y, color='blue', s=10, alpha=0.5, zorder=2)
      plt.xlim(0.48, 0.52)
      plt.xlabel('Phase')
      plt.ylabel('Relative flux');

### 3

https://heasarc.gsfc.nasa.gov/docs/tess/LightCurve-object-Tutorial.html

In [68]:
def folded_light_curve(n, tic_id,T0,period, catalog):
  data = catalog
  tess_id = data.loc(data['tic_id'] == tic_id)
  lightcurve = load_lightcurve(n,tess_id)
  duration = data['duration'][tess_id]
  cleaned_lightcurve = lightcurve.remove_outliers(sigma=20, sigma_upper=5)
  fold = cleaned_lightcurve.fold(period, t0=T0)
  norm_duration = (duration_hours / 24.0) / period
  mask = np.abs(fold.phase) < (norm_duration * 1.5)
  mask1 = np.in1d(cleaned_lightcurve.time, fold.time_original[mask])
  x, y = cleaned_lightcurve.flatten(return_trend=True, mask=mask1)
  return x.fold(period, t0=T0)

def phase_fold_lk(n, tic_id,T0,period,catalog, shift = False):
  data = catalog
  tess_id = data.loc(data['tic_id'] == tic_id)
  lc = folded_lightcurve(n, tic_id, T0,period,catalog)
  duration = stellar_params['duration'][tess_id]

  norm_duration = (duration / 24.0) / period
  mask = np.abs(lc.phase) < (0.5*norm_duration)
  zoomed = lc[mask]
  global_view = zoomed.bin(n_bins=201) 
  global_view = 2.0 * (global_view / np.abs(global_view.flux.min()))
  global_view = global_view.remove_nans()

  mask_2 = np.abs(lc.phase) < (2.0*norm_duration)
  zoom_lk = lc[mask_2]
  local_lk = zoom_lk.bin(n_bins=81) 
  local_lk = 2.0 * (local_lk / np.abs(local_lk.flux.min()))
  local_lk = local_lk.remove_nans()

  lc_2 = folded_lightcurve(n, tic_id, period, T0 + 0.25 * period)
  shifted = lc_2.bin(n_bins=201) 
  shifted = 2.0 * (half_phase / np.abs(half_phase.flux.min()))
  shifted = shifted.remove_nans()

  half_even = zoom_lk.bin(n_bins=81)  

  lc_3 = folded_lightcurve(n, tic_id, period, t0 - 0.5 * period)
  mask_3 = np.abs(lc_3.phase) < (2.0*norm_duration)
  zoom_lc = lc_3[mask_3]
  half_odd = zoom_lc.bin(n_bins=81) 

  time_net = np.concatenate((half_odd.time, half_even.time))
  flux_net = np.concatenate((half_odd.flux, half_even.flux))
  concatenated = lk.LightCurve(time_net, flux_net)
  half_phase = 2.0 * (concatenated / np.abs(concatenated.flux.min()))
  
  if shift == True:
    return global_view, local_lk, shifted
  else:
    return global_view, local_lk, half_phase

def final_lk(n, tic_id,catalog):
  data = stellar_params(n)
  try:
    period = data['period'][tess_id]
    T0 = data['T0'][tess_id]
    return phase_fold_lk(n, tic_id,T0,period,catalog)
  except:
    return None, None, None

### 4 Training

In [60]:
def label_split(n, files, catalog):
  positive_label = []
  negative_label = []
  tic_file = read_csv(n)
  for f in files:
    if f not in tic_file['Filename']:
      continue
    else:
      tess_id = tic_file['tic_id'][tic_file.loc(tic_file['Filename'] == f)]
      index = catalog.loc(catalog['tic_id'] == tess_id)
      if tess_id not in catalog['tic_id']:
        continue
      if catalog['Disposition'][tess_id] in ['KP', 'PC', 'CP']:
        positive_label.append(f)
      else:
        negative_label.append(f)
    return (positive_label, negative_label)

def train_test_split(files):
  random.seed(10)
  random.shuffle(files)
  file_len = len(files)
  train_data = files[:int(0.8*file_len)]
  test_data = files[int(0.8*file_len): int(0.9*file_len)]
  val_data = files[int(0.9*file_len):]
  return train_data, test_data, val_data

def balanced_batch_sampling(positives, negatives, batchsize=64):
        dataset = tf.data.Dataset.sample_from_datasets([positives, negatives], weights=[0.5, 0.5])
        dataset = dataset.batch(batchsize)
        return dataset

def dataset(files,stellar,batchsize=64):
  data = tf.data.TFRecordDataset(filenames = [files])
  def parsing(example_proto):
    features = {'tic_id': tf.io.FixedLenFeature([], tf.int64, default_value=0),'global_view': tf.io.VarLenFeature(tf.float32),
                                    'local_view': tf.io.VarLenFeature(tf.float32),'half_phase_view': tf.io.VarLenFeature(tf.float32),
                                    'disposition': tf.io.FixedLenFeature([], tf.string, default_value='')}
    stellar_transit_features = {i: tf.io.FixedLenFeature([1], tf.float32) for i in stellar}
    features.update(stellar_transit_features)
    parsing = tf.io.parse_single_example(example_proto, features)
    return parsing
  data = data.map(parsing)
  data = data.repeat()
  data = data.batch(batchsize)
  return dataset

https://www.tensorflow.org/api_docs/python/tf/data/Dataset#sample_from_datasets

In [15]:
ds2 = pd.read_csv('/content/drive/Othercomputers/My Laptop/ML Project/MAST/tces.csv')
ds2['Disposition'].value_counts()

J     12185
EB     2248
V       976
IS      816
PC      506
O         3
Name: Disposition, dtype: int64

In [16]:
# ds3 = pd.read_csv('/content/drive/Othercomputers/My Laptop/ML Project/MAST/exofop.csv')
# ds3['TESS Disposition'].value_counts()

In [17]:
ds4 = pd.read_csv('/content/drive/Othercomputers/My Laptop/ML Project/MAST/exo.csv')
ds4['User Disposition'].value_counts()

PC    495
FP      4
Name: User Disposition, dtype: int64

In [29]:
def cnn_model(global_shape,local_shape,half_phase, stellar_transit, bias):

  def nn(x, filter, kernel_size,act):
    return layers.Conv1D(filter, kernel_size,padding = 'same',activation = act)(x)

  def global_view_model(x):
    for i in [16,32,64,128,256]:
      x = nn(x,i,7,'relu')
      x = nn(x,i,7,'relu')
      x = layers.MaxPooling1D(5, 2)(x)
    return layers.Flatten()(x)

  def local_view_model(x):
    for i in [16,32]:
      x = nn(x,i,7,'relu')
      x = nn(x,i,7,'relu')
      x = layers.MaxPool1D(5,2)(x)
    return layers.Flatten()(x)

  global_in = tf.keras.Input(shape=global_shape)
  global_model = global_view_model(global_in)
  local_in = Input(shape=local_shape)
  local_model = local_view_model(local_in)
  half_phase_in = Input(shape=half_phase)
  half_phase_model = local_view_model(half_phase_in)

  combine = layers.Concatenate(axis=1)([global_model,local_model, half_phase_model])

  stellar_transit_in =[]
  for i in stellar_transit:
    y = Input(shape=(1,))
    combine = layers.Concatenate(axis=1)([combine, y])
    stellar_transit_in.append(y)
  # Fully connected layers
  model = keras.models.Sequential()
  model.add(layers.Dense(512,activation='relu'))
  for i in range(3):
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(512,activation='relu'))
  # Activation for output layer is sigmoid
  model.add(layers.Dense(1,activation='sigmoid',bias_initializer = keras.initializers.Constant(value=bias)))
  out = model(combine)

  final_model = keras.Model(inputs= [global_in,local_in, half_phase_in, stellar_transit_in], outputs = [out])
  adam = keras.optimizers.Adam(learning_rate=1e-5, epsilon= 1e-8)
  final_model.compile(loss= keras.losses.BinaryCrossentropy() , optimizer = adam, metrics = [keras.metrics.BinaryAccuracy()])
  
  return final_model


In [19]:
param_keys = { "Stellar Transit Features": ['temp_eff','radius','logg','Mass','luminosity','rho','depth','duration','depth_even','depth_odd','rp_rs'], 
          "Disposition": {"PC": 1,"KP": 1,"CP": 1, "EB": 0,"J": 0,"FP": 0, "O": 0,"V": 0, "IS": 0}}

In [66]:
def remove_nan(x):
  return tf.where(tf.math.is_nan(x),0, x)

def process(data,stellar):
  val = iter(data)
  item = next(val)
  global_array = tf.reshape(tf.sparse.to_dense(item['global_view']), [m, 201, 1])
  local_array = tf.reshape(tf.sparse.to_dense(item['local_view']), [m, 81, 1])
  half_array = tf.reshape(tf.sparse.to_dense(item['half_phase_view']), [m, 162, 1])
  glob = remove_nan(global_array)
  loca = remove_nan(local_array)
  half = remove_nan(half_array)
  array = {'global_view': glob, 'local_view': loca, 'half_phase_view': half}
  # disp = items['Disposition']
  st = {}
  for i in stellar:
    st[i]= remove_nan(item[i]).numpy()
    array.update({i: st[i]})
  return array
  

In [65]:
def training(files, param_keys):
  bias = np.log((506+495)/(12185+2248+976+816+3+4))
  stellar_transit = list(param_keys['Stellar Transit Features'])
  model = cnn_model((201, 1), (81, 1), (162, 1), stellar_transit, bias)
  train_dataset = pd.DataFrame()
  # stellar = pd.DataFrame()
  train_files = train_test_split(files)[0]
  test_files = train_test_split(files)[1]
  test_data = dataset(test_files,stellar_transit)
  valid_files = train_test_split(files)[2]
  validation = dataset(valid_files,stellar_transit)
  catalog = []
  for n in [6,21,22,23,24,25,26]:
    catalog.append(stellar_params(n))
  for n in [6,21,22,23,24,25,26]:
    pos_files = dataset(label_split(n, train_files, catalog)[0],stellar_transit)
    neg_files =  dataset(label_split(n, train_files, catalog)[1],stellar_transit)
    train_data = balanced_batch_sampling(pos_files, neg_files)
    train_dataset = pd.concat([train_dataset,train_data], ignore_index = True)
    # stellar = pd.concat([stellar,stellar_params], ignore_index =True)

  history = model.fit(process(train_dataset,stellar_transit),  epochs=50, batch_size= 64, validation_data=process(validation,stellar_transit))
  history = model.evaluate(process(test_data,stellar_transit))

In [58]:
files = []
for n in [6,21,22,23,24,25,26]:
  files.append(read_csv(n)['Filename'])
training(files,param_keys)

ok
Filename: /content/drive/Othercomputers/My Laptop/ML Project/MAST/Sector 21/tess2020020091053-s0021-0000000302696624-0167-s_lc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      44   ()      
  1  LIGHTCURVE    1 BinTableHDU    167   19694R x 20C   [D, E, J, E, E, E, E, E, E, J, D, E, D, E, D, E, D, E, E, E]   
  2  APERTURE      1 ImageHDU        49   (11, 11)   int32   
Searching with limb-darkening estimates using quadratic LD (a,b)= (0.3216, 0.2301)
Transit Least Squares TLS 1.0.31 (22 Nov 2021)
Creating model cache for 38 durations
Searching 18497 data points, 2833 periods from 0.829 to 13.673 days
Using all 2 CPU threads





  4%|▍         | 124/2833 periods | 12:06<4:24:24
 32%|███▏      | 3163/10010 periods | 09:22<20:18
  4%|▍         | 124/2833 periods | 12:06<4:24:29
 32%|███▏      | 3163/10010 periods | 09:23<20:18



  0%|          | 1/2833 periods | 00:04<3:27:27


  0%|          | 9/2833 periods | 00:04<17:12  


  1%|          | 19/2833 periods | 00:04<06:47


  1%|          | 35/2833 periods | 00:04<02:58


  2%|▏         | 51/2833 periods | 00:04<01:44


  2%|▏         | 67/2833 periods | 00:04<01:10


  3%|▎         | 83/2833 periods | 00:05<00:50


  3%|▎         | 97/2833 periods | 00:05<00:41


  4%|▍         | 115/2833 periods | 00:05<00:31


  5%|▍         | 131/2833 periods | 00:05<00:26


  5%|▌         | 148/2833 periods | 00:05<00:23


  6%|▌         | 164/2833 periods | 00:05<00:21


  6%|▋         | 180/2833 periods | 00:05<00:19


  7%|▋         | 197/2833 periods | 00:05<00:18


  8%|▊         | 213/2833 periods | 00:05<00:18


  8%|▊         | 229/2833 periods | 00:05<00:17




Searching for best T0 for period 10.76526 days


  4%|▍         | 124/2833 periods | 12:32<4:33:56
 32%|███▏      | 3163/10010 periods | 09:49<21:15


ok
Filename: /content/drive/Othercomputers/My Laptop/ML Project/MAST/Sector 21/tess2020020091053-s0021-0000000156495758-0167-s_lc.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      44   ()      
  1  LIGHTCURVE    1 BinTableHDU    167   19694R x 20C   [D, E, J, E, E, E, E, E, E, J, D, E, D, E, D, E, D, E, E, E]   
  2  APERTURE      1 ImageHDU        49   (11, 11)   int32   
Searching with limb-darkening estimates using quadratic LD (a,b)= (0.3785, 0.2075)
Transit Least Squares TLS 1.0.31 (22 Nov 2021)
Creating model cache for 38 durations
Searching 18554 data points, 6659 periods from 0.401 to 13.673 days
Using all 2 CPU threads


 90%|█████████ | 6008/6659 periods | 00:49<00:04Process ForkPoolWorker-25:
Process ForkPoolWorker-26:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.7/dist-packages/transitleastsquares/core.py", line 167, in search_period
    mean=1 - running_mean(patched_data, duration),
  File "/usr/local/lib/python3.7/dist-packages/transitleastsquares/helpers.py", line 73, in running_mean
    return (cumsum[width_sig

KeyboardInterrupt: ignored